# Hotels booking analysis

**Target**:
Analyze the behavior of users from different countries and the impact of having children on the success of the booking

**Main answers:**
- Which countries are the most successful in booking?

In [2]:
import pandas as pd

In [12]:
bookings = pd.read_csv('../../../data/01_module/02_part/2_bookings.csv', encoding='utf-8', sep=';')

In [73]:
bookings_head = bookings.head(7)
bookings_head

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


## Data preprocessing

In [145]:
rows, columns = bookings.shape
print(f'Rows: {rows}, columns: {columns}')

Rows: 119390, columns: 23


In [146]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 23 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   hotel                      119390 non-null  object 
 1   is_canceled                119390 non-null  int64  
 2   lead_time                  119390 non-null  int64  
 3   arrival_full_date          119390 non-null  object 
 4   arrival_date_year          119390 non-null  int64  
 5   arrival_date_month         119390 non-null  object 
 6   arrival_date_week_number   119390 non-null  int64  
 7   arrival_date_day_of_month  119390 non-null  int64  
 8   stays_in_weekend_nights    119390 non-null  int64  
 9   stays_in_week_nights       119390 non-null  int64  
 10  stays_total_nights         119390 non-null  int64  
 11  adults                     119390 non-null  int64  
 12  children                   119386 non-null  float64
 13  babies                     11

In [20]:
bookings.columns = df.columns.str.lower().str.replace(' ', '_')
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

## Group data by countury and filter by success bookings

In [153]:
bookings[bookings.is_canceled == 0].country.value_counts().head(5)

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: count, dtype: int64

## Getting the mean value of stays total nights in "City Hotel", "Resort Hotel"

In [155]:
bookings.groupby('hotel').stays_total_nights.mean().round(2)

hotel
City Hotel      2.98
Resort Hotel    4.32
Name: stays_total_nights, dtype: float64

## Discrepancy between the assigned number and the reserved one

In [50]:
bookings[bookings['assigned_room_type'] != bookings['reserved_room_type']].shape

(14917, 21)

## Group data by month and count bookings

### In 2016

In [69]:
bookings_2016_by_months = (
    bookings[bookings['arrival_date_year'] == 2016]
        .groupby('arrival_date_month', as_index=False)
        .agg({'hotel': 'count'})
        .rename(columns={'hotel': 'bookings'})
        .sort_values(by='bookings', ascending=False)
)

In [68]:
better_month_by_bookings = bookings_2016_by_months.iloc[0, :].arrival_date_month
better_month_by_bookings

'October'

### In 2017

In [70]:
bookings_2017_by_months = (
    bookings[bookings['arrival_date_year'] == 2017]
        .groupby('arrival_date_month', as_index=False)
        .agg({'hotel': 'count'})
        .rename(columns={'hotel': 'bookings'})
        .sort_values(by='bookings', ascending=False)
)

In [71]:
better_month_by_bookings = bookings_2017_by_months.iloc[0, :].arrival_date_month
better_month_by_bookings

'May'

## In which month of each year was the type "City Hotel" of hotel cancelled more often?

In [158]:
bookings.groupby(['arrival_date_year', 'arrival_date_month']).is_canceled.sum().groupby('arrival_date_year').nlargest(1)

arrival_date_year  arrival_date_year  arrival_date_month
2015               2015               September             2094
2016               2016               October               2514
2017               2017               May                   2762
Name: is_canceled, dtype: int64

In [159]:
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [101]:
bookings['total_kids'] = bookings['children'] + bookings['babies']

## Compare mean value of total kids in hotels: "City Hotel", "Resort Hotel"

In [161]:
bookings.groupby('hotel').total_kids.mean().round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

## Create churn rate of canceled booking users and how does having children affect this

In [108]:
bookings['has_kids'] = bookings['total_kids'] > 0

In [138]:
has_kids = bookings[bookings['has_kids']]
has_kids_canceled_bookings_score = has_kids[has_kids['is_canceled'] == 1].shape[0] / has_kids.shape[0]
has_kids_canceled_bookings_score

0.3492284612087441

In [142]:
has_not_kids = bookings[bookings['has_kids'] == False]
has_not_kids_canceled_bookings_score = has_not_kids[has_not_kids['is_canceled'] == 1].shape[0] / has_not_kids.shape[0]
has_not_kids_canceled_bookings_score`

0.37221283323338605

In [167]:
bookings[['has_kids', 'is_canceled']].value_counts() / bookings[['has_kids']].value_counts() * 100

has_kids  is_canceled
False     0              62.778717
          1              37.221283
True      0              65.077154
          1              34.922846
Name: count, dtype: float64